<a href="https://colab.research.google.com/github/mozey256/OSCAAR/blob/main/ML_Chemogenomics_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Orthology Based Design for Synergistic Drug Combinations Against ***Apsergillus fumigatus*** Using Chemogenomic and Drug Structural Data for ***Cryptococcus neoformans***

**Ainembabazi Moses 2022/HD07/2039U Makerere University IDI and ACE**

Chemogenomics data is a rich dataset for synergistic drug combination design but its not available for all fungal pathogens. Now lets solve this problem using Orthology Design

In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip



Defaulting to user installation because normal site-packages is not writeable

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bio 1.5.9 requires pooch, which is not installed.
pandas-profiling 2.7.1 requires visions[type_image_path]==0.4.1, but you have visions 0.7.6 which is incompatible.
pycaret 3.3.0 requires pandas<2.2.0, but you have pandas 2.2.1 which is incompatible.
sktime 0.27.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/17.8 MB ? eta -:--:--
     --------------------------------------- 0.0/17.8 MB 393.8 kB/s eta 0:00:46
     --------------------------------------- 0.1/17.8 MB 525.1 kB/s eta 0:00:34
     --------------------------------------- 0.1/17.8 MB 525.1 kB/s eta 0:00:34
     --------------------------------------- 0.2/17.8 MB 610.0 kB/s eta 0:00:29
     --------------------------------------- 0.2/17.8 MB 610.0 kB/s eta 0:00:29
     --------------------------------------- 0.2/17.8 MB 654.3 kB/s eta 0:00:27
      -------------------------------------- 0.3/17.8 MB 737.3 kB/s eta 0:00:24
      -------------------------------------- 0.3/17.8 MB 737.3 kB/s eta 0:00:24
      -------------------------------------- 0.3/17.8 MB 723.4 kB/s eta 0:00:25
      -------------------------------------- 0.4/17.8 MB 774.0 kB/s eta 0:00:23
      -------------------------------------- 0.4/17.8 M

In [2]:
!pip install rdkit

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pydantic-settings


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import all the required packages here

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
#---------------------- RDKit packages
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from pydantic_settings import BaseSettings
from pandas_profiling import ProfileReport
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit import DataStructs
import matplotlib.pyplot as plt
from rdkit.ML.Cluster import Butina
#------------------- progress bar
from tqdm import tqdm
#------------------- hide warning
import warnings
warnings.filterwarnings('ignore')

C:\Users\Moses\AppData\Local\Temp\ipykernel_15572\461858013.py:8: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


# CHEMOGENOMICS DATA

In [5]:
data = pd.read_csv('./Chemogenomic/Chemical-Genetics Dataset mmc3.csv')

check the loaded data


In [6]:
df = data


In [7]:
df.head()

,NAME,2aminobenzothiazole_conc1_T30,2aminobenzothiazole_conc2_T30,2aminobenzothiazole_conc3_T30,2hydroxyethylhydrazine_conc3_T30,3aminotriazole_conc1_T30,3aminotriazole_conc2_T30,3aminotriazole_conc3_T30,4hydroxytamoxifene_conc1_T30,4hydroxytamoxifene_conc2_T30,...,verrucarin_conc1_T30,verrucarin_conc2_T30,verrucarin_conc3_T30,ZnCl2_conc1_T30,ZnCl2_conc2_T30,ZnCl2_conc3_T30,average,stdev,-2.5 stdev,+2.5 stdev
0,CNAG_02695,1.056383,-27.976239,-12.390959,99.463196,-0.184176,-21.307448,-47.518637,13.259051,-7.443089,...,-107.722709,-4.442773,33.778516,89.765777,84.699476,80.812676,-7.219154,67.225201,-175.282156,160.843848
1,CNAG_06761,-9.030092,10.790969,26.735925,NaN,35.430513,33.511175,18.639025,-214.559477,NaN,...,NaN,NaN,NaN,-182.055423,-134.930253,-121.099541,-34.617671,107.448156,-303.238061,234.002719
2,CNAG_01862,-106.748572,-1.341236,-0.944822,NaN,-51.078138,23.378876,118.536041,NaN,-213.627304,...,-428.691317,-261.446249,NaN,-136.171531,-105.287600,-169.371448,-26.393266,130.335674,-352.232451,299.445918
3,CNAG_03664,-44.008538,-41.256120,-29.498142,-72.341975,12.642868,-83.957120,-110.532908,-24.765466,19.554120,...,-30.260754,-56.972615,-24.080345,105.303404,-66.122509,-105.019880,-6.469043,73.885705,-191.183305,178.245219
4,CNAG_01181,-3.578873,18.579128,-47.704215,123.927046,-71.786150,-29.939403,72.732075,19.046087,5.637677,...,-2.853405,94.638273,34.692855,-63.216421,-18.818562,6.887958,-2.804432,53.876986,-137.496897,131.888032


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452 entries, 0 to 1451
Columns: 444 entries, NAME to +2.5 stdev
dtypes: float64(443), object(1)
memory usage: 4.9+ MB


select every column name which represents a lower dosage of the drug which is conc3 and save it to a new file.

In [29]:
# Select all columns containing 'conc3' in their name
conc3_columns = [col for col in df.columns if 'conc3' in col]

# Add the specific fluconazole column to your list
selected_columns = conc3_columns + ['fluconazole_conc2_T30']

# Select the entries of these columns from the dataframe
selected_data = df[selected_columns]

# Save the selected data to a new CSV file
selected_data.to_csv('selected_dosage.csv', index=False)


In [9]:
df1_select = pd.read_csv('./Chemogenomic/selected_dosage.csv')

In [14]:
# Column name you want to check
column_to_check = '5-fluorocytosine_conc3_T30'

# Check if the column exists
if column_to_check in df1_select.columns:
    print(f"The column '{column_to_check}' exists in the DataFrame.")
else:
    print(f"The column '{column_to_check}' does not exist in the DataFrame.")

The column '5-fluorocytosine_conc3_T30' exists in the DataFrame.


In [18]:
import pandas as pd

# Read the original CSV file
data_select = df1_select
# Select the desired columns
columns_to_select = [
'2aminobenzothiazole_conc3_T30',
'2hydroxyethylhydrazine_conc3_T30',
'3aminotriazole_conc3_T30',
'4hydroxytamoxifene_conc3_T30',
'A23187_conc3_T30',
'abietic-acid_conc3_T30',
'acifluorofen-methyl_conc3_T30',
'aconitine_conc3_T30',
'agelasine_conc3_T30',
'alamethicin_conc3_T30',
'alexidine_conc3_T30',
'allantoin_conc3_T30',
'alternariol_conc3_T30',
'aluminum-sulfate_conc3_T30',
'amantadine_conc3_T30',
'amiodarone_conc3_T30',
'ammonium-persulfate_conc3_T30',
'amphotericinB_conc3_T30',
'antimycin_conc3_T30',
'apicidin_conc3_T30',
'artemesinin_conc3_T30',
'azide_conc3_T30',
'BaCl2_conc3_T30',
'bafilomycin_conc3_T30',
'BCS_conc3_T30',
'betulinic-acid_conc3_T30',
'bifonazole_conc3_T30',
'borate_conc3_T30',
'brefeldinA_conc3_T30',
'CaCl2_conc3_T30',
'caffeine_conc3_T30',
'calcofluor-white_conc3_T30',
'camptothecin_conc3_T30',
'castanospermine_conc3_T30',
'cerulenin_conc3_T30',
'chloroquine_conc3_T30',
'chlorpromazine_conc3_T30',
'chromium-III_conc3_T30',
'ciclopirox-olamine_conc3_T30',
'cisplatin_conc3_T30',
'climbazole_conc3_T30',
'clotrimazole_conc3_T30',
'colistin_conc3_T30',
'congo-red_conc3_T30',
'crystal-violet_conc3_T30',
'cycloheximide_conc3_T30',
'cyclosporin_conc3_T30',
'cyproconazole_conc3_T30',
'daphnetin_conc3_T30',
'deferoxamine_conc3_T30',
'desiperimine_conc3_T30',
'dyclonine_conc3_T30',
'emodin_conc3_T30',
'erlotinib_conc3_T30',
'fenpropimorph_conc3_T30',
'FK506_conc3_T30',
'fluconazole_conc2_T30',
'fluspirilene_conc3_T30',
'gallium-nitrate_conc3_T30',
'geldanamycin_conc3_T30',
'haloperidol_conc3_T30',
'hydrogen-peroxide_conc3_T30',
'hydroxyurea_conc3_T30',
'hygromycin_conc3_T30',
'imazilil_conc3_T30',
'itraconazole_conc3_T30',
'K252a_conc3_T30',
'LiCl_conc3_T30',
'lovastatin_conc3_T30',
'malachite-green_conc3_T30',
'manganese-sulfate_conc3_T30',
'mastaparan_conc3_T30',
'menadione_conc3_T30',
'menthol_conc3_T30',
'metavanadate_conc3_T30',
'methotrexate_conc3_T30',
'methylbenzethonium_conc3_T30',
'MG132_conc3_T30',
'miconazole_conc3_T30',
'mitomycin_conc3_T30',
'MMS-1_conc3_T30',
'myclobutanil_conc3_T30',
'mycophenolic-acid_conc3_T30',
'NA8_conc3_T30',
'NaCl_conc3_T30',
'NaNO2_conc3_T30',
'neomycin-sulfate_conc3_T30',
'nicotinamide_conc3_T30',
'nigericin_conc3_T30',
'nikkomycin_conc3_T30',
'NiSO4_conc3_T30',
'palmitic-acid_conc3_T30',
'pentamidine_isethionate_conc3_T30',
'phenanthroline_conc3_T30',
'polyoxinB_conc3_T30',
'Prussian-Blue_conc3_T30',
'quinic-acid_conc3_T30',
'rifamycin_conc3_T30',
'SDS_conc3_T30',
'selumenitib_conc3_T30',
'sertraline_conc3_T30',
'sodium-iodide_conc3_T30',
'sodium-molybdate_conc3_T30',
'sodium-selenite_conc3_T30',
'sodium-sulfite_conc3_T30',
'sorafenib_conc3_T30',
'STF-62247_conc3_T30',
'suloctodil_conc3_T30',
'tamoxifen-citrate_conc3_T30',
'tautomycin_conc3_T30',
'tellurite_conc3_T30',
'terbinafine_conc3_T30',
'thiabendazole_conc3_T30',
'thozonium-bromide_conc3_T30',
'tomatine_conc3_T30',
'trichostatinA_conc3_T30',
'trimethoprim_conc3_T30',
'tunicamycin_conc3_T30',
'usnic-acid_conc3_T30',
'valinomycin_conc3_T30',
'verrucarin_conc3_T30',
'ZnCl2_conc3_T30',]

selected_data = data[columns_to_select]

# Save the selected columns to a new CSV file
selected_data.to_csv('selected_columns_data.csv', index=False)

In [29]:
df_of_selected_d = pd.read_csv("./selected_columns_data.csv")


In [30]:
columns = list(df_of_selected_d.columns)
columns

['2aminobenzothiazole_conc3_T30',
 '2hydroxyethylhydrazine_conc3_T30',
 '3aminotriazole_conc3_T30',
 '4hydroxytamoxifene_conc3_T30',
 'A23187_conc3_T30',
 'abietic-acid_conc3_T30',
 'acifluorofen-methyl_conc3_T30',
 'aconitine_conc3_T30',
 'agelasine_conc3_T30',
 'alamethicin_conc3_T30',
 'alexidine_conc3_T30',
 'allantoin_conc3_T30',
 'alternariol_conc3_T30',
 'aluminum-sulfate_conc3_T30',
 'amantadine_conc3_T30',
 'amiodarone_conc3_T30',
 'ammonium-persulfate_conc3_T30',
 'amphotericinB_conc3_T30',
 'antimycin_conc3_T30',
 'apicidin_conc3_T30',
 'artemesinin_conc3_T30',
 'azide_conc3_T30',
 'BaCl2_conc3_T30',
 'bafilomycin_conc3_T30',
 'BCS_conc3_T30',
 'betulinic-acid_conc3_T30',
 'bifonazole_conc3_T30',
 'borate_conc3_T30',
 'brefeldinA_conc3_T30',
 'CaCl2_conc3_T30',
 'caffeine_conc3_T30',
 'calcofluor-white_conc3_T30',
 'camptothecin_conc3_T30',
 'castanospermine_conc3_T30',
 'cerulenin_conc3_T30',
 'chloroquine_conc3_T30',
 'chlorpromazine_conc3_T30',
 'chromium-III_conc3_T30',


In [31]:
df_of_selected_d.describe()

,2aminobenzothiazole_conc3_T30,2hydroxyethylhydrazine_conc3_T30,3aminotriazole_conc3_T30,4hydroxytamoxifene_conc3_T30,A23187_conc3_T30,abietic-acid_conc3_T30,acifluorofen-methyl_conc3_T30,aconitine_conc3_T30,agelasine_conc3_T30,alamethicin_conc3_T30,...,thiabendazole_conc3_T30,thozonium-bromide_conc3_T30,tomatine_conc3_T30,trichostatinA_conc3_T30,trimethoprim_conc3_T30,tunicamycin_conc3_T30,usnic-acid_conc3_T30,valinomycin_conc3_T30,verrucarin_conc3_T30,ZnCl2_conc3_T30
count,1447.000000,1447.000000,1446.000000,1451.000000,1448.000000,1446.000000,1450.000000,1443.000000,1448.000000,1448.000000,...,1448.000000,1451.000000,1449.000000,1372.000000,1447.000000,1451.000000,1446.000000,1447.000000,1446.000000,1452.000000
mean,-3.259671,-1.900895,1.927553,-2.586860,0.520220,0.910867,-0.469433,-1.171001,-0.236123,-2.941660,...,-1.329431,-0.102921,-3.095922,1.344102,0.174904,-2.540051,-1.293033,-0.713006,-2.688197,-1.297769
std,33.123578,42.666550,45.864301,28.960188,24.496190,23.932250,24.699677,26.891972,26.886021,27.525268,...,38.997733,21.438528,37.268917,21.267259,26.667256,57.230765,21.456906,30.789559,26.565339,27.502170
min,-371.385938,-425.235728,-395.478995,-400.171641,-198.099914,-376.301691,-400.885844,-437.180055,-365.125912,-210.499068,...,-188.729555,-146.387238,-383.835539,-358.366759,-118.804684,-429.860597,-102.415940,-438.528773,-440.556178,-242.398371
25%,-18.127331,-16.889843,-16.588563,-10.213033,-12.211059,-10.387951,-12.686126,-9.723610,-9.733992,-14.952158,...,-23.934240,-10.606191,-20.975288,-7.455343,-14.378870,-27.554736,-13.152196,-11.037369,-10.487528,-13.126277
50%,-1.724132,-1.746717,2.761458,-0.764871,-0.261006,0.938718,0.208273,-0.116559,1.205677,-2.711213,...,-0.769562,-0.819104,-0.711218,0.769675,1.184842,0.077744,-0.564275,0.845338,0.298391,-0.138352
75%,14.602861,18.173445,21.539027,9.303977,12.484823,12.810821,12.671962,10.026371,11.416434,10.240174,...,22.268803,10.154084,17.046668,9.435652,16.146399,29.160638,10.842204,12.765992,8.953357,11.167245
max,108.770371,223.063183,268.118561,75.530561,118.597040,119.521476,88.409488,142.516313,155.004199,152.066598,...,134.360324,101.798766,154.291009,186.841095,124.446323,321.110502,82.523704,161.446366,134.951335,158.757308


In [32]:
# Column name you want to check
column_to_check = '5-fluorocytosine_conc3_T30'

# Check if the column exists
if column_to_check in df_of_selected_d.columns:
    print(f"The column '{column_to_check}' exists in the DataFrame.")
else:
    print(f"The column '{column_to_check}' does not exist in the DataFrame.")

The column '5-fluorocytosine_conc3_T30' exists in the DataFrame.


# Check for missing data and  handle it

In [33]:

missing_data = df_of_selected_d.isnull().sum()

# Print the count of missing values for each column
print("Missing data count:")
print(missing_data)


Missing data count:
2aminobenzothiazole_conc3_T30       5
2hydroxyethylhydrazine_conc3_T30    5
3aminotriazole_conc3_T30            6
4hydroxytamoxifene_conc3_T30        1
A23187_conc3_T30                    4
                                   ..
tunicamycin_conc3_T30               1
usnic-acid_conc3_T30                6
valinomycin_conc3_T30               5
verrucarin_conc3_T30                6
ZnCl2_conc3_T30                     0
Length: 123, dtype: int64


Fill the missing data with the mean


In [35]:
# Fill missing values with the mean of the column
df_of_selected_d.fillna(df_of_selected_d.mean(), inplace=True)

df_of_selected_d.to_csv('filledmissing_chemo_data.csv', index=False)

In [36]:
df2 = pd.read_csv('./filledmissing_chemo_data.csv')

check for missing data again


In [37]:
missing_data = df2.isnull().sum()

# Print the count of missing values for each column
print("Missing data count:")
print(missing_data)

Missing data count:
2aminobenzothiazole_conc3_T30       0
2hydroxyethylhydrazine_conc3_T30    0
3aminotriazole_conc3_T30            0
4hydroxytamoxifene_conc3_T30        0
A23187_conc3_T30                    0
                                   ..
tunicamycin_conc3_T30               0
usnic-acid_conc3_T30                0
valinomycin_conc3_T30               0
verrucarin_conc3_T30                0
ZnCl2_conc3_T30                     0
Length: 123, dtype: int64


Check for Duplicates and remove  them if they are available

In [38]:
# Check for duplicates
print(df2.duplicated().sum())

# Remove duplicates if necessary
#df.drop_duplicates(inplace=True)

0


# Visualize the data to understand distributions, relationships, and trends.

In [ ]:
# Distribution of each feature
# for column in df2.columns:
#     sns.histplot(df2[column].dropna())
#     plt.title(f'Distribution of {column}')
#     plt.show()

# Pairplot to visualize relationships between features
# sns.pairplot(df2)
# plt.show()

# Correlation heatmap
# corr = df2.corr()
# sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Correlation Heatmap')
# plt.show()

In [36]:
profile = ProfileReport(df2, title= 'c.neo chemogenomics_reprpt', html={'style':{'full_width':True}})

In [ ]:
profile = ProfileReport(df2, title="Pandas Profiling Report")

profile.to_file("outputReport.html")

# To display the report in the notebook
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]